In [1]:
import torch
import numpy as np

In [2]:
# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device:", device)

# Move a tensor to GPU (if available)
x = torch.arange(5).to(device)
print("x is on:", x.device)

Running on device: cpu
x is on: cpu


In [3]:
# load result data set from exercise 2
data = np.load("fashion_mnist_garments.npz")

x_garments = data["x_garments"]
y_garments = data["y_garments"]
x_others = data["x_others"]
y_others = data["y_others"]

In [4]:
# convert to torch tensors

x_garments = torch.from_numpy(x_garments).float()
y_garments = torch.from_numpy(y_garments).float()   

x_others = torch.from_numpy(x_others).float()
y_others = torch.from_numpy(y_others).float()

In [5]:
# (a) Define numeric input

def input_features(images):
    left_symmetry = images[:, :, :14].mean(dim = (1, 2))  
    right_symmetry = images[:, :, 14:].mean(dim = (1, 2))  
    x1 = torch.abs(left_symmetry - right_symmetry)

    not_zero = images > 0   
    a = not_zero.any(dim=1).sum(dim=1).float()
    b = not_zero.any(dim=2).sum(dim=1).float()
    x2 = a / b

    return torch.stack((x1, x2), dim=1)

In [6]:
# (b) Normalize features

def standardize_features(features):
    mean = features.mean(dim=0)
    std = features.std(dim=0)
    return (features - mean) / std

In [7]:
# (c) Weighted sum of inputs

W1 = torch.randn(2,3) * 0.1 # hidden layer input
W2 = torch.randn(3,1) * 0.1 # hidden layer output
b1 = torch.randn(3) * 0.1 
b2 = torch.randn(1) * 0.1

# print("W1:", W1)
# print("b1:", b1)    

features = input_features(x_garments)
x_hat = standardize_features(features)
print(x_hat)

h = x_hat.float() @ W1 + b1

print('the shape of h is:', h.shape , '\n' '\n' 'first 5 entries ''\n', h[0:5])

tensor([[ 1.7306,  0.7230],
        [-0.1403, -1.7090],
        [-0.8700, -0.4930],
        ...,
        [ 1.2429, -0.2498],
        [-0.4805, -0.0066],
        [ 0.9053,  0.4798]])
the shape of h is: torch.Size([35000, 3]) 

first 5 entries 
 tensor([[-0.1941, -0.1145,  0.5731],
        [-0.1751, -0.0537, -0.1415],
        [-0.0541, -0.0378, -0.2363],
        [-0.1235, -0.0408, -0.2551],
        [-0.0307, -0.0512, -0.0577]])


In [8]:
# (d) Apply non-linear activation
h_activated = torch.sigmoid(h)

# (e) Weighted sum of hidden layer
y = (h_activated @ W2) + b2

# (f) Output activation
final_output = torch.sigmoid(y)
print(final_output[0:5])

tensor([[0.4816],
        [0.4817],
        [0.4817],
        [0.4817],
        [0.4817]])
